In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.api import qqplot
from sklearn.metrics import mean_squared_error as mse
%matplotlib inline

In [2]:
# Load data from CSV file
dta = pd.read_csv('data.csv')

# Convert 'Date' column to datetime type
dta['Date'] = pd.to_datetime(dta['Date'])

# Set 'Date' column as index
dta.set_index('Date', inplace=True)

# Drop 'YEAR' column if it exists
if 'YEAR' in dta.columns:
    del dta['YEAR']

# Print the loaded data
print(dta)

                    spx   spx_ret   spx_vol
Date                                       
1994-01-06   467.119995       NaN       NaN
1994-01-07   469.899994  0.595136  0.595136
1994-01-10   475.269989  1.142795  1.142795
1994-01-11   474.130005 -0.239860  0.239860
1994-01-12   474.170013  0.008438  0.008438
...                 ...       ...       ...
2019-08-26  2878.379883  1.098299  1.098299
2019-08-27  2869.159912 -0.320318  0.320318
2019-08-28  2887.939941  0.654548  0.654548
2019-08-29  2924.580078  1.268729  1.268729
2019-08-30  2926.459961  0.064279  0.064279

[6692 rows x 3 columns]


In [4]:
# Split into train and test
percentage = 0.6
series = dta['spx'].tolist()
size = int(len(series) * 0.66)
train, test = series[0:size], series[size:len(series)]

In [9]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing

# ARIMA model
model = ARIMA(train, order=(9, 0, 0))
model_fit = model.fit()

# Autocorrelation
acf_1 = model_fit.autocorr()
plt.plot(acf_1)
plt.show()

ModuleNotFoundError: No module named 'distutils'

In [ ]:
# Partial Autocorrelation
pacf_1 = model_fit.pacf()
plt.plot(pacf_1)
plt.show()

In [ ]:
# Rolling Forecast
predicted1, resid_test = [], []
history = train
for t in range(len(test)):
    model = ARIMA(history, order=(9, 0, 0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    resid_test.append(test[t] - output[0])
    predicted1.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

# Residual Diagnostics
print('Test MSE: %.3f' % mean_squared_error(test, predicted1))
residuals = model_fit.resid
plt.plot(residuals)
plt.show()
plt.hist(residuals, bins=20)
plt.show()

In [ ]:
# Hybrid Model
window_size = 50

def make_model(window_size):
    model = Sequential()
    model.add(Dense(50, input_dim=window_size, activation="tanh"))
    model.add(Dense(25, activation="tanh"))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

model = make_model(window_size)
min_max_scaler = preprocessing.MinMaxScaler()
train_scaled = min_max_scaler.fit_transform(np.array(train).reshape(-1, 1))

train_X, train_Y = [], []
for i in range(0, len(train_scaled) - window_size):
    train_X.append(train_scaled[i:i + window_size])
    train_Y.append(train_scaled[i + window_size])

new_train_X, new_train_Y = [], []
for i in train_X:
    new_train_X.append(i.reshape(-1))
for i in train_Y:
    new_train_Y.append(i.reshape(-1))
new_train_X = np.array(new_train_X)
new_train_Y = np.array(new_train_Y)

model.fit(new_train_X, new_train_Y, epochs=500, batch_size=512, validation_split=0.05)